In [180]:
# Dependencies
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import numpy as np

def scrape(*args):
    city_search=set(args)
    return city_search

In [181]:
lista=scrape("New York", "San Francisco", "Os")
search_city_list= [city.replace(" ", "+") for city in lista] * 10
search_city_list.sort()
print(search_city_list)

['New+York', 'New+York', 'New+York', 'New+York', 'New+York', 'New+York', 'New+York', 'New+York', 'New+York', 'New+York', 'Os', 'Os', 'Os', 'Os', 'Os', 'Os', 'Os', 'Os', 'Os', 'Os', 'San+Francisco', 'San+Francisco', 'San+Francisco', 'San+Francisco', 'San+Francisco', 'San+Francisco', 'San+Francisco', 'San+Francisco', 'San+Francisco', 'San+Francisco']


In [171]:
max_results_city = 100
step = 10
city_search = ["New+York", "San+Francisco"]
job_search = "data+analyst"

url_list = [] # will hold list of urls for each job
search_city_list= [city.replace("+", " ") for city in city_search] * max_results_city
search_city_list.sort()
print(search_city_list)

['New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'Ne

In [172]:
for city in city_search:
    for start in range(0, max_results_city, step):
        search_url = "http://www.indeed.com/jobs?q="+str(job_search)+"&l="+str(city)+"&start="+str(start)
        results_page = requests.get(search_url)
        #browser.visit(search_url)
        time.sleep(2)  #ensuring at least 1 second between page grabs
        #html = browser.html
        soup = BeautifulSoup(results_page.text, "lxml",)

        h2 = soup.find_all('h2', class_="jobtitle")
        for stuff in h2:
            href = stuff.find('a')['href']
            url = "http://www.indeed.com/"+str(href)
            url_list.append(url)
            
print("Done")

Done


In [173]:
job_company = []
job_desc = []
job_location = []
job_title = []
job_salary = []
job_url = []

In [174]:
for url in url_list:
    job_page = requests.get(url,)
    time.sleep(1) 
    
    job_soup = BeautifulSoup(job_page.text, "lxml")
    
    title=company=location=job_summary=""
        
    title_object = job_soup.find("b", class_="jobtitle")
    if title_object is not None:
        title = title_object.text
    job_title.append(title)

    company_object = job_soup.find("span", class_="company")
    if company_object is not None:
        company = company_object.text
    job_company.append(company)

    location_object = job_soup.find("span", class_="location")
    if location_object is not None:
        location = location_object.text
    job_location.append(location)

    job_summary_object = job_soup.find("span", id="job_summary")
    if job_summary_object is not None:
        job_summary = job_summary_object.get_text()
        
    job_desc.append(job_summary)
    
    job_url.append(url)


print(len(job_title))
print(len(job_desc))
print(len(job_location))
print(len(job_company))
print(len(job_url))
print(len(search_city_list))




200
200
200
200
200
200


In [175]:
example_df = pd.DataFrame({"Title": job_title,
                           "Company": job_company,
                           "Location": job_location,
                           "Description": job_desc,
                           #"Salary" : job_salary
                           "Link": job_url,
                           "Search_City": search_city_list})

example_df = example_df[["Title","Company", "Location","Description", "Search_City", "Link"]]
example_df.index = np.arange(1, len(example_df) + 1)

In [176]:
example_df.head()

,Title,Company,Location,Description,Search_City,Link
1,Data Analyst Intern,QBE,New York State,To assist Development Leads for Proof of Conce...,New York,http://www.indeed.com//rc/clk?jk=be1de9ec0bd2e...
2,Data Analyst,Acacia Technical Services,New York State,Title: Data Analyst\nLocation either NYC or Pa...,New York,http://www.indeed.com//rc/clk?jk=f9bc2efbe1dfb...
3,Data Analyst,The New York Times,"New York, NY",About the position:\nWirecutter is seeking a D...,New York,http://www.indeed.com//rc/clk?jk=2f4ab0a8ac3e0...
4,"Data Analyst, Optimization",The New York Times,"New York, NY",The New York Times is seeking a Data Analyst t...,New York,http://www.indeed.com//rc/clk?jk=9490bd919be4a...
5,Data Analyst,Morgan Stanley,"New York, NY 10032",he Data Management Organization (DMO) is leadi...,New York,http://www.indeed.com//rc/clk?jk=1c57c8959e16f...


In [177]:
# Save to new csv file
output = os.path.join('..', '03_Data', 'Job_Data.csv')
print(output)
example_df.to_csv(output, header=True, index=True, index_label='Job_Id', encoding='utf-8-sig')

../03_Data/Job_Data.csv
